In [2]:
import pandas as pd
import os
import json
import re
import glob


In [27]:
data = {}
categories = []

for filename in os.listdir("../data/processed_csv"):
    if filename.endswith(".csv"):
        data[filename[:-4]] = pd.read_csv("../data/processed_csv/" + filename)
        categories.append(filename[:-4])


### Storing Attributes in JSON under ../data/processed_json {for all cat individually}

#### Just to look what's in the attributes

In [38]:
for cat in categories:
    json_list = []
    skip = 0
    for index, row in data[cat].iterrows():
        i = row['style_attributes']
        i = i.replace("'", '"').encode('utf-8').decode('unicode_escape')
        try: 
            json_obj = json.loads(i)
            json_obj['sno'] = row['sno']
            json_list.append(json_obj)
        except:
            skip+=1
            continue
        
    print(f"Writing {cat}.json skipping {skip} rows out of {len(data[cat])}")
    with open(f'../data/processed_json/{cat}.json', 'w') as f:
        json.dump(json_list, f)


Writing Dresses.json skipping 0 rows out of 7795
Writing Shirts.json skipping 162 rows out of 9106
Writing Kurtis.json skipping 216 rows out of 14031
Writing Sneakers.json skipping 257 rows out of 4660
Writing Saree.json skipping 174 rows out of 7305
Writing Jeans.json skipping 158 rows out of 21366
Writing Bathroom.json skipping 1426 rows out of 9546
Writing Earrings.json skipping 683 rows out of 5112
Writing Tshirts.json skipping 36 rows out of 5721
Writing Watches.json skipping 3076 rows out of 9856


### Frequncy and Refined Frequecy Analysis

In [22]:
data_properties = {}
data_refined_properties = {}
# Iterate through each JSON file

for filename in os.listdir("../data/processed_json"):
    if filename.endswith(".json"):
        cat_properties = {}
        with open(f"../data/processed_json/{filename}", 'r') as f:
            data = json.load(f)
            for item in data:
                for key, value in item.items():
                    if key in cat_properties:
                        cat_properties[key] += 1
                    else:
                        cat_properties[key] = 1
        
        cat_refined_properties = {}
        for key, value in cat_properties.items():
            if key == 'sno':
                continue
            if value>=10:
                cat_refined_properties[key] = value

        #sort the cat_refined_properties based on the values
        cat_properties = {k:v for k,v in sorted(cat_properties.items(), key=lambda item:item[1], reverse=True)}
        cat_refined_properties = {k:v for k,v in sorted(cat_refined_properties.items(), key=lambda item:item[1], reverse=True)}
        
        data_properties[filename.split('.')[0]] = cat_properties
        data_refined_properties[filename.split('.')[0]] = cat_refined_properties


In [21]:
with open('../data/attributes_frequency.json', 'w') as f:
    json.dump(data_properties, f)

with open('../data/attributes_refined_frequency.json', 'w') as f:
    json.dump(data_refined_properties, f)